In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
np.random.seed(777)
import math
import keras
import keras.backend as K
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, Dropout, concatenate
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix, roc_auc_score, auc, f1_score
from keras.regularizers import l2
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.regularizers import l2, l1

from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import SeparableConv2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["axes.grid"] = False
plt.rcParams.update({'font.size': 20})

Using TensorFlow backend.


In [3]:
train_dir = 'data/train/'
test_dir = "data/test/"

# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Dense201_Xception_Inceptionv3"

In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.10.0
dim_ordering: tf


In [5]:
batch_size = 512
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
# for root,dirs,files in os.walk(train_dir):
#     print (root, len(files))

# print("*"*30)
# for root,dirs,files in os.walk(test_dir):
#     print (root, len(files))

In [7]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [8]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    validation_split= 0.2,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 319152 images belonging to 6 classes.
Found 79784 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [9]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 319152
nb_validation_samples: 79784
nb_test_samples: 13578

predict_size_train: 624
predict_size_validation: 156
predict_size_test: 27

 num_classes: 6


In [10]:
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))


# reset_keras_tf_session()

In [ ]:
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications.inception_v3 import InceptionV3


input_tensor = Input(shape = input_shape)  

base_model1=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model3=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

x3 = base_model3.output
x3 = GlobalAveragePooling2D()(x3)

merge = concatenate([x1, x2, x3])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

In [ ]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[572].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[586].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[602].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[636].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[679].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[702].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[722].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[738].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[749].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[783].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[847].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[911].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[975].output
c17 = GlobalAveragePooling2D()(c17) 

c18 = model.layers[1042].output
c18 = GlobalAveragePooling2D()(c18) 

c19 = model.layers[1070].output
c19 = GlobalAveragePooling2D()(c19) 

c20 = model.layers[1106].output
c20 = GlobalAveragePooling2D()(c20) 

c21 = model.layers[1125].output
c21 = GlobalAveragePooling2D()(c21) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, c21])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [ ]:
bottleneck_final_model.summary()

In [ ]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [ ]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [11]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [12]:
# dropout_rate = 0.5
model = Sequential()

model.add(Dense(4096, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))
model.add(Dropout(0.25))
model.add(Dense(512, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))

model.add(Dense(128, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))

model.add(Dense(256, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))

model.add(Dense(16, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))

model.add(Dense(32, activation="relu", kernel_regularizer=l2(1e-07), bias_regularizer=l2(0.001), activity_regularizer=l1(1e-06)))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation="softmax"))

adam_opt=Adam(lr = 1e-06, beta_1=0.6, beta_2=0.999)

model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels,
                    epochs=1000,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2)

with open(extracted_features_dir+'history_'+model_name+'.txt','w') as f:
    f.write(str(history.history))
    
top_model_path = os.path.join(extracted_features_dir, 'model_'+model_name+'_model.h5')
top_model_weights_path = os.path.join(extracted_features_dir, 'model_'+model_name+'_weights.h5')    
    
model.save_weights(top_model_weights_path)
model.save(top_model_path)

Train on 319152 samples, validate on 79784 samples
Epoch 1/1000
 - 21s - loss: 2.1467 - acc: 0.3569 - val_loss: 1.7319 - val_acc: 0.4811
Epoch 2/1000
 - 17s - loss: 1.6381 - acc: 0.5165 - val_loss: 1.4432 - val_acc: 0.6187
Epoch 3/1000
 - 17s - loss: 1.4667 - acc: 0.5669 - val_loss: 1.3142 - val_acc: 0.6441
Epoch 4/1000
 - 17s - loss: 1.3810 - acc: 0.5889 - val_loss: 1.2275 - val_acc: 0.6767
Epoch 5/1000
 - 17s - loss: 1.3174 - acc: 0.6075 - val_loss: 1.1535 - val_acc: 0.7104
Epoch 6/1000
 - 17s - loss: 1.2646 - acc: 0.6228 - val_loss: 1.0960 - val_acc: 0.7528
Epoch 7/1000
 - 17s - loss: 1.2188 - acc: 0.6360 - val_loss: 1.0458 - val_acc: 0.7767
Epoch 8/1000
 - 17s - loss: 1.1794 - acc: 0.6479 - val_loss: 1.0026 - val_acc: 0.7963
Epoch 9/1000
 - 17s - loss: 1.1437 - acc: 0.6581 - val_loss: 0.9631 - val_acc: 0.8074
Epoch 10/1000
 - 17s - loss: 1.1126 - acc: 0.6669 - val_loss: 0.9298 - val_acc: 0.8168
Epoch 11/1000
 - 17s - loss: 1.0839 - acc: 0.6761 - val_loss: 0.8966 - val_acc: 0.8230
E

Epoch 95/1000
 - 17s - loss: 0.4420 - acc: 0.8913 - val_loss: 0.3014 - val_acc: 0.9416
Epoch 96/1000
 - 17s - loss: 0.4393 - acc: 0.8925 - val_loss: 0.2988 - val_acc: 0.9419
Epoch 97/1000
 - 17s - loss: 0.4373 - acc: 0.8922 - val_loss: 0.2977 - val_acc: 0.9422
Epoch 98/1000
 - 17s - loss: 0.4341 - acc: 0.8935 - val_loss: 0.2962 - val_acc: 0.9423
Epoch 99/1000
 - 17s - loss: 0.4320 - acc: 0.8935 - val_loss: 0.2932 - val_acc: 0.9434
Epoch 100/1000
 - 17s - loss: 0.4297 - acc: 0.8942 - val_loss: 0.2930 - val_acc: 0.9427
Epoch 101/1000
 - 17s - loss: 0.4270 - acc: 0.8950 - val_loss: 0.2890 - val_acc: 0.9438
Epoch 102/1000
 - 17s - loss: 0.4252 - acc: 0.8957 - val_loss: 0.2879 - val_acc: 0.9443
Epoch 103/1000
 - 17s - loss: 0.4214 - acc: 0.8968 - val_loss: 0.2860 - val_acc: 0.9442
Epoch 104/1000
 - 17s - loss: 0.4193 - acc: 0.8972 - val_loss: 0.2854 - val_acc: 0.9446
Epoch 105/1000
 - 17s - loss: 0.4168 - acc: 0.8976 - val_loss: 0.2835 - val_acc: 0.9449
Epoch 106/1000
 - 17s - loss: 0.4150 

 - 17s - loss: 0.2846 - acc: 0.9387 - val_loss: 0.1918 - val_acc: 0.9641
Epoch 189/1000
 - 17s - loss: 0.2845 - acc: 0.9395 - val_loss: 0.1920 - val_acc: 0.9637
Epoch 190/1000
 - 17s - loss: 0.2840 - acc: 0.9394 - val_loss: 0.1906 - val_acc: 0.9641
Epoch 191/1000
 - 17s - loss: 0.2825 - acc: 0.9402 - val_loss: 0.1934 - val_acc: 0.9632
Epoch 192/1000
 - 17s - loss: 0.2789 - acc: 0.9412 - val_loss: 0.1891 - val_acc: 0.9646
Epoch 193/1000
 - 17s - loss: 0.2795 - acc: 0.9413 - val_loss: 0.1899 - val_acc: 0.9640
Epoch 194/1000
 - 17s - loss: 0.2787 - acc: 0.9414 - val_loss: 0.1882 - val_acc: 0.9647
Epoch 195/1000
 - 17s - loss: 0.2770 - acc: 0.9424 - val_loss: 0.1873 - val_acc: 0.9649
Epoch 196/1000
 - 17s - loss: 0.2761 - acc: 0.9420 - val_loss: 0.1860 - val_acc: 0.9653
Epoch 197/1000
 - 17s - loss: 0.2752 - acc: 0.9429 - val_loss: 0.1853 - val_acc: 0.9652
Epoch 198/1000
 - 17s - loss: 0.2747 - acc: 0.9434 - val_loss: 0.1854 - val_acc: 0.9654
Epoch 199/1000
 - 17s - loss: 0.2722 - acc: 0.9

Epoch 282/1000
 - 17s - loss: 0.2027 - acc: 0.9632 - val_loss: 0.1452 - val_acc: 0.9737
Epoch 283/1000
 - 17s - loss: 0.2023 - acc: 0.9632 - val_loss: 0.1460 - val_acc: 0.9735
Epoch 284/1000
 - 17s - loss: 0.2025 - acc: 0.9626 - val_loss: 0.1436 - val_acc: 0.9738
Epoch 285/1000
 - 17s - loss: 0.2018 - acc: 0.9632 - val_loss: 0.1451 - val_acc: 0.9739
Epoch 286/1000
 - 17s - loss: 0.2011 - acc: 0.9634 - val_loss: 0.1454 - val_acc: 0.9732
Epoch 287/1000
 - 17s - loss: 0.2001 - acc: 0.9634 - val_loss: 0.1434 - val_acc: 0.9739
Epoch 288/1000
 - 17s - loss: 0.2004 - acc: 0.9633 - val_loss: 0.1438 - val_acc: 0.9742
Epoch 289/1000
 - 17s - loss: 0.1993 - acc: 0.9637 - val_loss: 0.1427 - val_acc: 0.9744
Epoch 290/1000
 - 17s - loss: 0.1983 - acc: 0.9642 - val_loss: 0.1444 - val_acc: 0.9740
Epoch 291/1000
 - 17s - loss: 0.1980 - acc: 0.9638 - val_loss: 0.1436 - val_acc: 0.9741
Epoch 292/1000
 - 17s - loss: 0.1985 - acc: 0.9636 - val_loss: 0.1434 - val_acc: 0.9739
Epoch 293/1000
 - 17s - loss: 0.

 - 17s - loss: 0.1568 - acc: 0.9730 - val_loss: 0.1227 - val_acc: 0.9786
Epoch 376/1000
 - 17s - loss: 0.1557 - acc: 0.9737 - val_loss: 0.1211 - val_acc: 0.9789
Epoch 377/1000
 - 17s - loss: 0.1544 - acc: 0.9738 - val_loss: 0.1242 - val_acc: 0.9785
Epoch 378/1000
 - 17s - loss: 0.1547 - acc: 0.9734 - val_loss: 0.1230 - val_acc: 0.9784
Epoch 379/1000
 - 17s - loss: 0.1544 - acc: 0.9735 - val_loss: 0.1227 - val_acc: 0.9786
Epoch 380/1000
 - 17s - loss: 0.1546 - acc: 0.9736 - val_loss: 0.1215 - val_acc: 0.9785
Epoch 381/1000
 - 17s - loss: 0.1540 - acc: 0.9736 - val_loss: 0.1227 - val_acc: 0.9789
Epoch 382/1000
 - 17s - loss: 0.1531 - acc: 0.9737 - val_loss: 0.1224 - val_acc: 0.9791
Epoch 383/1000
 - 17s - loss: 0.1531 - acc: 0.9735 - val_loss: 0.1224 - val_acc: 0.9791
Epoch 384/1000
 - 17s - loss: 0.1525 - acc: 0.9738 - val_loss: 0.1229 - val_acc: 0.9787
Epoch 385/1000
 - 17s - loss: 0.1525 - acc: 0.9740 - val_loss: 0.1199 - val_acc: 0.9794
Epoch 386/1000
 - 17s - loss: 0.1521 - acc: 0.9

...

 - 17s - loss: 0.1063 - acc: 0.9844 - val_loss: 0.1036 - val_acc: 0.9836
Epoch 563/1000
 - 17s - loss: 0.1069 - acc: 0.9842 - val_loss: 0.1046 - val_acc: 0.9833
Epoch 564/1000
 - 17s - loss: 0.1061 - acc: 0.9843 - val_loss: 0.1016 - val_acc: 0.9834
Epoch 565/1000
 - 17s - loss: 0.1073 - acc: 0.9839 - val_loss: 0.1012 - val_acc: 0.9840
Epoch 566/1000
 - 17s - loss: 0.1060 - acc: 0.9841 - val_loss: 0.1034 - val_acc: 0.9839
Epoch 567/1000
 - 17s - loss: 0.1067 - acc: 0.9843 - val_loss: 0.1015 - val_acc: 0.9837
Epoch 568/1000
 - 17s - loss: 0.1058 - acc: 0.9847 - val_loss: 0.1025 - val_acc: 0.9839
Epoch 569/1000
 - 17s - loss: 0.1056 - acc: 0.9843 - val_loss: 0.1038 - val_acc: 0.9835
Epoch 570/1000
 - 17s - loss: 0.1055 - acc: 0.9845 - val_loss: 0.1024 - val_acc: 0.9836
Epoch 571/1000
 - 17s - loss: 0.1050 - acc: 0.9842 - val_loss: 0.1021 - val_acc: 0.9841
Epoch 572/1000
 - 17s - loss: 0.1054 - acc: 0.9844 - val_loss: 0.1021 - val_acc: 0.9837
Epoch 573/1000
 - 17s - loss: 0.1060 - acc: 0.9

Epoch 656/1000
 - 17s - loss: 0.0938 - acc: 0.9865 - val_loss: 0.0988 - val_acc: 0.9849
Epoch 657/1000
 - 17s - loss: 0.0942 - acc: 0.9864 - val_loss: 0.0988 - val_acc: 0.9845
Epoch 658/1000
 - 17s - loss: 0.0930 - acc: 0.9866 - val_loss: 0.0979 - val_acc: 0.9845
Epoch 659/1000
 - 17s - loss: 0.0919 - acc: 0.9871 - val_loss: 0.0995 - val_acc: 0.9843
Epoch 660/1000
 - 17s - loss: 0.0927 - acc: 0.9867 - val_loss: 0.0989 - val_acc: 0.9851
Epoch 661/1000
 - 17s - loss: 0.0933 - acc: 0.9863 - val_loss: 0.0993 - val_acc: 0.9846
Epoch 662/1000
 - 17s - loss: 0.0930 - acc: 0.9863 - val_loss: 0.0992 - val_acc: 0.9843
Epoch 663/1000
 - 17s - loss: 0.0928 - acc: 0.9867 - val_loss: 0.0987 - val_acc: 0.9848
Epoch 664/1000
 - 17s - loss: 0.0923 - acc: 0.9866 - val_loss: 0.0977 - val_acc: 0.9850
Epoch 665/1000
 - 17s - loss: 0.0924 - acc: 0.9869 - val_loss: 0.0970 - val_acc: 0.9847
Epoch 666/1000
 - 17s - loss: 0.0927 - acc: 0.9867 - val_loss: 0.1007 - val_acc: 0.9847
Epoch 667/1000
 - 17s - loss: 0.

 - 17s - loss: 0.0846 - acc: 0.9878 - val_loss: 0.0967 - val_acc: 0.9853
Epoch 750/1000
 - 17s - loss: 0.0837 - acc: 0.9881 - val_loss: 0.0949 - val_acc: 0.9857
Epoch 751/1000
 - 17s - loss: 0.0829 - acc: 0.9881 - val_loss: 0.0943 - val_acc: 0.9859
Epoch 752/1000
 - 17s - loss: 0.0829 - acc: 0.9885 - val_loss: 0.0961 - val_acc: 0.9851
Epoch 753/1000
 - 17s - loss: 0.0834 - acc: 0.9880 - val_loss: 0.0973 - val_acc: 0.9851
Epoch 754/1000
 - 17s - loss: 0.0837 - acc: 0.9879 - val_loss: 0.0945 - val_acc: 0.9856
Epoch 755/1000
 - 17s - loss: 0.0833 - acc: 0.9880 - val_loss: 0.0943 - val_acc: 0.9854
Epoch 756/1000
 - 17s - loss: 0.0838 - acc: 0.9880 - val_loss: 0.0950 - val_acc: 0.9854
Epoch 757/1000
 - 17s - loss: 0.0837 - acc: 0.9879 - val_loss: 0.0947 - val_acc: 0.9859
Epoch 758/1000
 - 17s - loss: 0.0825 - acc: 0.9883 - val_loss: 0.0945 - val_acc: 0.9855
Epoch 759/1000
 - 17s - loss: 0.0841 - acc: 0.9877 - val_loss: 0.0935 - val_acc: 0.9856
Epoch 760/1000
 - 17s - loss: 0.0830 - acc: 0.9

Epoch 843/1000
 - 17s - loss: 0.0759 - acc: 0.9891 - val_loss: 0.0936 - val_acc: 0.9860
Epoch 844/1000
 - 17s - loss: 0.0762 - acc: 0.9892 - val_loss: 0.0932 - val_acc: 0.9861
Epoch 845/1000
 - 17s - loss: 0.0770 - acc: 0.9890 - val_loss: 0.0966 - val_acc: 0.9857
Epoch 846/1000
 - 17s - loss: 0.0774 - acc: 0.9888 - val_loss: 0.0945 - val_acc: 0.9860
Epoch 847/1000
 - 17s - loss: 0.0764 - acc: 0.9890 - val_loss: 0.0953 - val_acc: 0.9852
Epoch 848/1000
 - 17s - loss: 0.0757 - acc: 0.9894 - val_loss: 0.0916 - val_acc: 0.9863
Epoch 849/1000
 - 17s - loss: 0.0762 - acc: 0.9890 - val_loss: 0.0912 - val_acc: 0.9863
Epoch 850/1000
 - 17s - loss: 0.0751 - acc: 0.9895 - val_loss: 0.0933 - val_acc: 0.9856
Epoch 851/1000
 - 17s - loss: 0.0756 - acc: 0.9891 - val_loss: 0.0945 - val_acc: 0.9860
Epoch 852/1000
 - 17s - loss: 0.0756 - acc: 0.9894 - val_loss: 0.0932 - val_acc: 0.9860
Epoch 853/1000
 - 17s - loss: 0.0762 - acc: 0.9889 - val_loss: 0.0952 - val_acc: 0.9858
Epoch 854/1000
 - 17s - loss: 0.

 - 17s - loss: 0.0703 - acc: 0.9903 - val_loss: 0.0884 - val_acc: 0.9869
Epoch 937/1000
 - 17s - loss: 0.0703 - acc: 0.9903 - val_loss: 0.0899 - val_acc: 0.9865
Epoch 938/1000
 - 17s - loss: 0.0710 - acc: 0.9900 - val_loss: 0.0936 - val_acc: 0.9861
Epoch 939/1000
 - 17s - loss: 0.0711 - acc: 0.9900 - val_loss: 0.0930 - val_acc: 0.9863
Epoch 940/1000
 - 17s - loss: 0.0711 - acc: 0.9899 - val_loss: 0.0895 - val_acc: 0.9867
Epoch 941/1000
 - 17s - loss: 0.0702 - acc: 0.9901 - val_loss: 0.0907 - val_acc: 0.9863
Epoch 942/1000
 - 17s - loss: 0.0702 - acc: 0.9902 - val_loss: 0.0909 - val_acc: 0.9866
Epoch 943/1000
 - 17s - loss: 0.0704 - acc: 0.9901 - val_loss: 0.0926 - val_acc: 0.9864
Epoch 944/1000
 - 17s - loss: 0.0717 - acc: 0.9897 - val_loss: 0.0906 - val_acc: 0.9866
Epoch 945/1000
 - 17s - loss: 0.0702 - acc: 0.9903 - val_loss: 0.0895 - val_acc: 0.9869
Epoch 946/1000
 - 17s - loss: 0.0709 - acc: 0.9898 - val_loss: 0.0906 - val_acc: 0.9867
Epoch 947/1000
 - 17s - loss: 0.0700 - acc: 0.9

In [13]:
preds = model.predict(test_data)

predictions = [i.argmax() for i in preds]
y_true = [i.argmax() for i in test_labels]
cm = confusion_matrix(y_pred=predictions, y_true=y_true)

print('Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

Accuracy 0.9451318309029312


In [14]:
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity*100 )

Specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', Specificity*100 )

Sensitivity :  99.93004547044421
Specificity :  99.58677685950413
